In [40]:
import plaidml.keras
plaidml.keras.install_backend()

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab

import statsmodels.api as sm
import statsmodels.tsa.api as smt
from statsmodels.tsa.arima_model import ARIMA
import statsmodels.tsa.stattools as ts
from statsmodels.tsa.stattools import adfuller

from fbprophet import Prophet

import math

# import pyflux as pf

import warnings
warnings.filterwarnings(action='once')

import itertools

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import RidgeCV
from sklearn.metrics import mean_squared_error

from sklearn.model_selection import TimeSeriesSplit

from pandas.plotting import autocorrelation_plot

import re

import sys
import os

from functools import reduce

import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Activation, Dropout, Input, LSTM
#from keras.layers import Concatbenate
from keras.utils import np_utils
from keras.utils.np_utils import to_categorical
from keras.utils.data_utils import get_file
from keras.preprocessing.text import Tokenizer
from keras.utils.vis_utils import model_to_dot, plot_model
from keras.datasets import imdb, reuters
from keras.preprocessing import sequence
from keras.optimizers import SGD, RMSprop
from keras.models import load_model

import datetime as dt

from sklearn.preprocessing import StandardScaler, MinMaxScaler

import pickle

from numpy.random import seed

from tensorflow import set_random_seed

In [41]:
seed(2019)
set_random_seed(2019)

In [42]:
df_combined = pd.read_pickle('./processed_data/df_combined.pickle')

df_combined.tail(2)

,ds,y,vix,gold
12666,2019-06-01 22:00:00,8568.81,30.447,3561.58
12667,2019-06-01 23:00:00,8560.63,30.447,3569.34


In [43]:
df_daily = df_combined[df_combined.ds.dt.hour == 0]

In [44]:
df_daily.reset_index(inplace=True, drop = True)

In [45]:
df_daily.head(3)

,ds,y,vix,gold
0,2017-02-08,1054.03,18.879,2732.00
1,2017-02-09,1059.04,18.619,2703.51
2,2017-02-10,986.42,18.165,2793.34


In [46]:
def split_train_test_chronological(df, ratio = 0.9, use_ratio = True, index = 1000):
    
    '''
    Input is a dataframe, and a ratio. Splits dataframe into 2 dataframes chronologically.
    Returns first dataframe up to the index of the length of the input dataframe times the input ratio, 
    and returns second dataframe of remaining elements.
    use_ratio is a flag, wether ratio should be used or indicies instead.
    
    df = input dataframe
    ratio = ratio to be used for splitting
    use_ratio = if True, use ratio, 
    index = index to split input dataframe on
    
    '''
    if use_ratio:
        size = len(df) * ratio
        size_round = round(size)

        df_train = df[0:(size_round)]
        df_test = df[size_round:]
    else:
        df_train = df[0:(index)]
        df_test = df[index:]
    return df_train, df_test

In [47]:
def make_variables(df, lags, look_back, predict_window):
    
    '''
    In order to use keras LSTM, we need to convert the input into a keras-friendly input.
    
    df = input dataframe
    lags = number of lags
    look_back = number of preceding elements to be considered
    predict_window = size of window for predictions
    
    '''
    
    features = lags + 1
    start = look_back
    stop = len(df) - lags - predict_window

    lstm_in_X = np.zeros(shape=(stop-start, look_back+1, features))
    lstm_in_Y = np.zeros(shape=(stop-start, look_back+1))

    iter_list = [num for num in range(look_back+1)][::-1]
    for i in range(start, stop):
        for index, j in enumerate(iter_list):
            X = df[i - j : i - j + lags + 1, -1]
            lstm_in_X[i - start, index] = np.ravel(X)
            Y = df[i - j + lags + 1, -1]
            lstm_in_Y[i-start, index] = Y
            
    return lstm_in_X, lstm_in_Y, features

In [48]:
def train_lstm(train_X, train_Y, lags, look_back, predict_window, lstm_nodes,
               dense_layers, dropout = 0.1, loss_type = 'hinge', optimizer_type = 'adam',
               number_epochs = 300, batch_size = 24, ):
    
    
    model = Sequential()
    model.add(LSTM(lstm_nodes, input_shape=(look_back+1, features)))
    model.add(Dropout(dropout))
    for nodes in dense_layers:
        model.add(Dense(nodes))
        model.add(Dropout(dropout))
    model.add(Dense(look_back + 1))
    model.compile(loss=loss_type, optimizer= optimizer_type)
    model.fit(train_X, train_Y, epochs=number_epochs, batch_size = batch_size, verbose = 1)
    
    pred_Y_train = model.predict(train_X)
    predictions = pred_Y_train[:,-1]
    actuals = train_Y[:,-1]
    
    print(model.summary())
    
    return model, dataset, train_X, train_Y, predictions, actuals

In [49]:
def make_rolling_window(model, train_x, train_y, test_x, test_y, batch_size = 218, epochs = 2):
    predictions_test = []
    actuals_test = []
    yhats = []
    ys = []
    
    dimension = train_x.shape[2]
    
    for i in range(0, len(test_y) - 1):
        model.fit(train_x, train_y, 
                    epochs= epochs, 
                    batch_size = batch_size, 
                    validation_data=(test_x[i].reshape(1,1,dimension), (test_y[i].reshape(1 ,))),
                    verbose=2,
#                     callbacks=[earlystopper],
                    shuffle=False)
        pred_Y_test = model.predict(test_x[i].reshape(1,1,dimension))
        train_x = np.concatenate((train_x, (test_x[i].reshape(1 , 1 , dimension))))

        train_y = np.concatenate((train_y, (test_y[i].reshape(1 ,))))
        predict_test = pred_Y_test[-1,-1]
        actual_test = train_y[-1]
        predictions_test.append(predict_test)
        actuals_test.append(actual_test)
        yhats.append(pred_Y_test)
        ys.append(test_y[i])
        
    return predictions_test, actuals_test, yhats, ys

In [50]:
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    '''
    
    
    '''
    n_vars = 1 if type(data) is list else data.shape[1]
    df = pd.DataFrame(data)
    cols, names = list(), list()
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
    # put it all together
    agg = pd.concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg

In [59]:
# load dataset
values = df_daily.drop('ds', axis = 1).values
# values = df_combined.drop('ds', axis = 1).values
# integer encode direction
# ensure all data is float
values = values.astype('float32')
# normalize features
# scaler = MinMaxScaler()
scaler = StandardScaler()
scaled = scaler.fit_transform(values)
# frame as supervised learning
reframed = series_to_supervised(scaled, 7, 1)
reframed.head()
# reframed_2 = series_to_supervised(values, 7, 1)
# reframed_2.head()

# drop columns we don't want to predict
y = reframed.iloc[:,-12].values
X = reframed.drop('var1(t)', axis =1).values
# split into train and test sets

n_train_days = 500
n_test_days = 10
train_X = X[:n_train_days,:]
train_y = y[:n_train_days]

val_X= X[n_train_days:-n_test_days,]
val_y= y[n_train_days:-n_test_days]


test_X = X[-n_test_days:,:]
test_y = y[-n_test_days:]



# reshape input to be 3D [samples, timesteps, features]
train_X = train_X.reshape((train_X.shape[0], 1, train_X.shape[1]))
val_X = val_X.reshape((val_X.shape[0], 1, val_X.shape[1]))
test_X = test_X.reshape((test_X.shape[0], 1, test_X.shape[1]))

train_and_val_X = np.concatenate((train_X, val_X), axis = 0)

train_and_val_y = np.concatenate((train_y, val_y), axis = 0)

print(train_X.shape, train_y.shape,val_X.shape, val_y.shape ,test_X.shape, test_y.shape)

(500, 1, 23) (500,) (11, 1, 23) (11,) (10, 1, 23) (10,)


In [60]:
scaler = StandardScaler()
scaled = scaler.fit_transform(np.array(df_daily.y[:n_train_days]).reshape(-1,1))

In [61]:
LSTM_model_daily_1 = Sequential()
LSTM_model_daily_1.add(LSTM(64, activation='relu', input_shape=(train_X.shape[1], train_X.shape[2]), dropout=0.20,recurrent_dropout=0.20))
# LSTM_model_daily_1.add(LSTM(128))
LSTM_model_daily_1.add(Dense(32))
LSTM_model_daily_1.add(Dense(32))
# LSTM_model_daily_1.add(LSTM(16, activation='relu'))
LSTM_model_daily_1.add(Dense(1))
LSTM_model_daily_1.compile(optimizer='adam', loss='mse')


In [62]:
history_daily_1 = LSTM_model_daily_1.fit(train_X, train_y, 
                    epochs= 200, 
                    batch_size = 32, 
                    validation_data=(val_X, val_y),
                    verbose=2,
#                     callbacks=[earlystopper],
                    shuffle=False)

Train on 500 samples, validate on 11 samples
Epoch 1/200
 - 0s - loss: 0.7781 - val_loss: 0.3023
Epoch 2/200
 - 0s - loss: 0.2174 - val_loss: 0.0447
Epoch 3/200
 - 0s - loss: 0.1019 - val_loss: 0.0073
Epoch 4/200
 - 0s - loss: 0.0669 - val_loss: 0.0072
Epoch 5/200
 - 0s - loss: 0.0505 - val_loss: 0.0056
Epoch 6/200
 - 0s - loss: 0.0442 - val_loss: 0.0033
Epoch 7/200
 - 0s - loss: 0.0496 - val_loss: 0.0084
Epoch 8/200
 - 0s - loss: 0.0454 - val_loss: 0.0020
Epoch 9/200
 - 0s - loss: 0.0444 - val_loss: 0.0224
Epoch 10/200
 - 0s - loss: 0.0436 - val_loss: 0.0022
Epoch 11/200
 - 0s - loss: 0.0473 - val_loss: 0.0152
Epoch 12/200
 - 0s - loss: 0.0595 - val_loss: 0.0104
Epoch 13/200
 - 0s - loss: 0.0719 - val_loss: 0.0425
Epoch 14/200
 - 0s - loss: 0.0858 - val_loss: 0.0479
Epoch 15/200
 - 0s - loss: 0.1341 - val_loss: 0.0389
Epoch 16/200
 - 0s - loss: 0.0535 - val_loss: 0.0407
Epoch 17/200
 - 0s - loss: 0.0719 - val_loss: 0.0742
Epoch 18/200
 - 0s - loss: 0.0581 - val_loss: 0.0184
Epoch 19/2

Epoch 154/200
 - 0s - loss: 0.0406 - val_loss: 0.0075
Epoch 155/200
 - 0s - loss: 0.0297 - val_loss: 0.0200
Epoch 156/200
 - 0s - loss: 0.0331 - val_loss: 0.0054
Epoch 157/200
 - 0s - loss: 0.0308 - val_loss: 0.0183
Epoch 158/200
 - 0s - loss: 0.0323 - val_loss: 0.0016
Epoch 159/200
 - 0s - loss: 0.0344 - val_loss: 0.0183
Epoch 160/200
 - 0s - loss: 0.0510 - val_loss: 0.0017
Epoch 161/200
 - 0s - loss: 0.0388 - val_loss: 0.0116
Epoch 162/200
 - 0s - loss: 0.0419 - val_loss: 0.0051
Epoch 163/200
 - 0s - loss: 0.0300 - val_loss: 0.0295
Epoch 164/200
 - 0s - loss: 0.0346 - val_loss: 0.0097
Epoch 165/200
 - 0s - loss: 0.0331 - val_loss: 0.0277
Epoch 166/200
 - 0s - loss: 0.0473 - val_loss: 0.0112
Epoch 167/200
 - 0s - loss: 0.0400 - val_loss: 0.0475
Epoch 168/200
 - 0s - loss: 0.0643 - val_loss: 0.0068
Epoch 169/200
 - 0s - loss: 0.0364 - val_loss: 0.0168
Epoch 170/200
 - 0s - loss: 0.0342 - val_loss: 0.0059
Epoch 171/200
 - 0s - loss: 0.0306 - val_loss: 0.0176
Epoch 172/200
 - 0s - loss: 

In [63]:
res_1 = make_rolling_window(LSTM_model_daily_1, train_X, train_y, val_X, val_y, batch_size = 32, epochs = 2)

Train on 500 samples, validate on 1 samples
Epoch 1/2
 - 0s - loss: 0.0469 - val_loss: 0.0300
Epoch 2/2
 - 0s - loss: 0.0592 - val_loss: 0.0096
Train on 501 samples, validate on 1 samples
Epoch 1/2
 - 0s - loss: 0.0406 - val_loss: 0.0184
Epoch 2/2
 - 0s - loss: 0.0731 - val_loss: 0.0099
Train on 502 samples, validate on 1 samples
Epoch 1/2
 - 0s - loss: 0.0385 - val_loss: 0.0406
Epoch 2/2
 - 0s - loss: 0.0513 - val_loss: 1.6457e-04
Train on 503 samples, validate on 1 samples
Epoch 1/2
 - 0s - loss: 0.0364 - val_loss: 0.0528
Epoch 2/2
 - 0s - loss: 0.0517 - val_loss: 7.1228e-04
Train on 504 samples, validate on 1 samples
Epoch 1/2
 - 0s - loss: 0.0324 - val_loss: 0.0392
Epoch 2/2
 - 0s - loss: 0.0463 - val_loss: 2.9368e-04
Train on 505 samples, validate on 1 samples
Epoch 1/2
 - 0s - loss: 0.0294 - val_loss: 0.0449
Epoch 2/2
 - 0s - loss: 0.0287 - val_loss: 0.0051
Train on 506 samples, validate on 1 samples
Epoch 1/2
 - 0s - loss: 0.0272 - val_loss: 0.0120
Epoch 2/2
 - 0s - loss: 0.0285

In [64]:
predicts_1 = scaler.inverse_transform(np.array(res_1[0]))
actuals_1 = scaler.inverse_transform(np.array(res_1[1]))
rmse = math.sqrt(mean_squared_error(predicts_1, actuals_1))
print('Validation RMSE: %.3f' % rmse)

Validation RMSE: 247.169


In [65]:
res_test = make_rolling_window(LSTM_model_daily_1, train_and_val_X, train_and_val_y, test_X, test_y, batch_size = 32, epochs = 2)


Train on 511 samples, validate on 1 samples
Epoch 1/2
 - 0s - loss: 0.0270 - val_loss: 0.0116
Epoch 2/2
 - 0s - loss: 0.0311 - val_loss: 2.1841e-05
Train on 512 samples, validate on 1 samples
Epoch 1/2
 - 0s - loss: 0.0274 - val_loss: 0.0013
Epoch 2/2
 - 0s - loss: 0.0340 - val_loss: 0.0100
Train on 513 samples, validate on 1 samples
Epoch 1/2
 - 0s - loss: 0.0340 - val_loss: 0.0159
Epoch 2/2
 - 0s - loss: 0.0586 - val_loss: 0.0039
Train on 514 samples, validate on 1 samples
Epoch 1/2
 - 0s - loss: 0.0492 - val_loss: 0.1260
Epoch 2/2
 - 0s - loss: 0.0710 - val_loss: 0.0076
Train on 515 samples, validate on 1 samples
Epoch 1/2
 - 0s - loss: 0.0544 - val_loss: 0.1655
Epoch 2/2
 - 0s - loss: 0.0752 - val_loss: 0.0095
Train on 516 samples, validate on 1 samples
Epoch 1/2
 - 0s - loss: 0.0572 - val_loss: 0.0789
Epoch 2/2
 - 0s - loss: 0.1047 - val_loss: 0.0065
Train on 517 samples, validate on 1 samples
Epoch 1/2
 - 0s - loss: 0.0483 - val_loss: 0.0793
Epoch 2/2
 - 0s - loss: 0.0767 - val_l

In [66]:
predicts_1 = scaler.inverse_transform(np.array(res_test[0]))
actuals_1 = scaler.inverse_transform(np.array(res_test[1]))
rmse = math.sqrt(mean_squared_error(predicts_1, actuals_1))
print('TEST RMSE: %.3f' % rmse)

TEST RMSE: 271.686


In [67]:
LSTM_model_daily_2 = Sequential()
LSTM_model_daily_2.add(LSTM(128, activation='relu', input_shape=(train_X.shape[1], train_X.shape[2]), dropout=0.20,recurrent_dropout=0.20))
# LSTM_model_daily_2.add(LSTM(128))
LSTM_model_daily_2.add(Dense(64))
LSTM_model_daily_2.add(Dense(64))
# LSTM_model_daily_2.add(LSTM(16, activation='relu'))
LSTM_model_daily_2.add(Dense(1))
LSTM_model_daily_2.compile(optimizer='adam', loss='mse')

In [68]:
history_daily_2 = LSTM_model_daily_2.fit(train_X, train_y, 
                    epochs= 200, 
                    batch_size = 32, 
                    validation_data=(val_X, val_y),
                    verbose=2,
#                     callbacks=[earlystopper],
                    shuffle=False)

Train on 500 samples, validate on 11 samples
Epoch 1/200
 - 7s - loss: 0.8799 - val_loss: 0.2917
Epoch 2/200
 - 0s - loss: 0.0909 - val_loss: 0.0472
Epoch 3/200
 - 0s - loss: 0.2167 - val_loss: 0.1519
Epoch 4/200
 - 0s - loss: 0.2226 - val_loss: 0.0128
Epoch 5/200
 - 0s - loss: 0.0560 - val_loss: 0.0202
Epoch 6/200
 - 0s - loss: 0.0629 - val_loss: 0.0392
Epoch 7/200
 - 0s - loss: 0.0662 - val_loss: 0.0421
Epoch 8/200
 - 0s - loss: 0.0488 - val_loss: 0.0244
Epoch 9/200
 - 0s - loss: 0.0566 - val_loss: 0.0456
Epoch 10/200
 - 0s - loss: 0.0551 - val_loss: 0.0445
Epoch 11/200
 - 0s - loss: 0.0652 - val_loss: 0.0853
Epoch 12/200
 - 0s - loss: 0.0553 - val_loss: 0.1338
Epoch 13/200
 - 0s - loss: 0.0743 - val_loss: 0.0484
Epoch 14/200
 - 0s - loss: 0.0665 - val_loss: 0.0919
Epoch 15/200
 - 0s - loss: 0.1194 - val_loss: 0.0563
Epoch 16/200
 - 0s - loss: 0.1092 - val_loss: 0.1037
Epoch 17/200
 - 0s - loss: 0.1998 - val_loss: 0.0119
Epoch 18/200
 - 0s - loss: 0.0497 - val_loss: 0.0173
Epoch 19/2

Epoch 154/200
 - 0s - loss: 0.0269 - val_loss: 0.0038
Epoch 155/200
 - 0s - loss: 0.0242 - val_loss: 0.0105
Epoch 156/200
 - 0s - loss: 0.0254 - val_loss: 0.0033
Epoch 157/200
 - 0s - loss: 0.0269 - val_loss: 0.0125
Epoch 158/200
 - 0s - loss: 0.0388 - val_loss: 0.0031
Epoch 159/200
 - 0s - loss: 0.0666 - val_loss: 0.0016
Epoch 160/200
 - 0s - loss: 0.0481 - val_loss: 0.0024
Epoch 161/200
 - 0s - loss: 0.0701 - val_loss: 0.0069
Epoch 162/200
 - 0s - loss: 0.0517 - val_loss: 0.0515
Epoch 163/200
 - 0s - loss: 0.0822 - val_loss: 0.0319
Epoch 164/200
 - 0s - loss: 0.0630 - val_loss: 0.0796
Epoch 165/200
 - 0s - loss: 0.1149 - val_loss: 0.0382
Epoch 166/200
 - 0s - loss: 0.0837 - val_loss: 0.0919
Epoch 167/200
 - 0s - loss: 0.1752 - val_loss: 0.0033
Epoch 168/200
 - 0s - loss: 0.1624 - val_loss: 0.1554
Epoch 169/200
 - 0s - loss: 0.4254 - val_loss: 0.0287
Epoch 170/200
 - 0s - loss: 0.0832 - val_loss: 0.0998
Epoch 171/200
 - 0s - loss: 0.2615 - val_loss: 0.1373
Epoch 172/200
 - 0s - loss: 

In [70]:
res_2 = make_rolling_window(LSTM_model_daily_2, train_X, train_y, val_X, val_y, batch_size = 32, epochs = 2)

Train on 500 samples, validate on 1 samples
Epoch 1/2
 - 2s - loss: 0.0248 - val_loss: 9.7547e-05
Epoch 2/2
 - 0s - loss: 0.0270 - val_loss: 8.6474e-05
Train on 501 samples, validate on 1 samples
Epoch 1/2
 - 3s - loss: 0.0278 - val_loss: 1.5940e-04
Epoch 2/2
 - 0s - loss: 0.0301 - val_loss: 8.1111e-04
Train on 502 samples, validate on 1 samples
Epoch 1/2
 - 3s - loss: 0.0296 - val_loss: 7.7227e-05
Epoch 2/2
 - 0s - loss: 0.0296 - val_loss: 0.0121
Train on 503 samples, validate on 1 samples
Epoch 1/2
 - 3s - loss: 0.0331 - val_loss: 3.5895e-04
Epoch 2/2
 - 0s - loss: 0.0288 - val_loss: 0.0302
Train on 504 samples, validate on 1 samples
Epoch 1/2
 - 3s - loss: 0.0340 - val_loss: 0.0061
Epoch 2/2
 - 0s - loss: 0.0281 - val_loss: 0.0125
Train on 505 samples, validate on 1 samples
Epoch 1/2
 - 3s - loss: 0.0338 - val_loss: 1.1766e-04
Epoch 2/2
 - 0s - loss: 0.0330 - val_loss: 0.0606
Train on 506 samples, validate on 1 samples
Epoch 1/2
 - 3s - loss: 0.0437 - val_loss: 0.0358
Epoch 2/2
 - 0

In [71]:
predicts_2 = scaler.inverse_transform(np.array(res_2[0]))
actuals_2 = scaler.inverse_transform(np.array(res_2[1]))
rmse = math.sqrt(mean_squared_error(predicts_2, actuals_2))
print('Validation RMSE: %.3f' % rmse)

Validation RMSE: 515.363


In [72]:
res_test_2 = make_rolling_window(LSTM_model_daily_2, train_and_val_X, train_and_val_y, test_X, test_y, batch_size = 32, epochs = 2)


Train on 511 samples, validate on 1 samples
Epoch 1/2
 - 1s - loss: 0.0321 - val_loss: 0.0027
Epoch 2/2
 - 0s - loss: 0.0240 - val_loss: 0.0342
Train on 512 samples, validate on 1 samples
Epoch 1/2
 - 0s - loss: 0.0253 - val_loss: 0.0034
Epoch 2/2
 - 0s - loss: 0.0229 - val_loss: 0.0144
Train on 513 samples, validate on 1 samples
Epoch 1/2
 - 3s - loss: 0.0223 - val_loss: 2.2793e-04
Epoch 2/2
 - 0s - loss: 0.0311 - val_loss: 0.0129
Train on 514 samples, validate on 1 samples
Epoch 1/2
 - 2s - loss: 0.0217 - val_loss: 0.0015
Epoch 2/2
 - 0s - loss: 0.0257 - val_loss: 0.0204
Train on 515 samples, validate on 1 samples
Epoch 1/2
 - 2s - loss: 0.0298 - val_loss: 4.7395e-04
Epoch 2/2
 - 0s - loss: 0.0675 - val_loss: 0.0054
Train on 516 samples, validate on 1 samples
Epoch 1/2
 - 2s - loss: 0.0487 - val_loss: 0.0126
Epoch 2/2
 - 0s - loss: 0.0566 - val_loss: 0.0038
Train on 517 samples, validate on 1 samples
Epoch 1/2
 - 3s - loss: 0.0478 - val_loss: 0.0974
Epoch 2/2
 - 0s - loss: 0.0540 - v

In [73]:
predicts_2 = scaler.inverse_transform(np.array(res_test_2[0]))
actuals_2 = scaler.inverse_transform(np.array(res_test_2[1]))
rmse = math.sqrt(mean_squared_error(predicts_2, actuals_2))
print('TEST RMSE: %.3f' % rmse)

TEST RMSE: 442.742
